In [1]:
%run "Common setup.ipynb"

In [2]:
def create_inactive(num_inactive):
    
    num_vars = 3 + num_inactive
    
    inactive_problem = {
      'num_vars': num_vars,
      'names': ['x1', 'x2', 'x3'] + ['x{}'.format(i) for i in range(4, 4+num_inactive)],
      'bounds': [[-np.pi, np.pi]]*num_vars,
      'nominal': [0.1] * num_vars
    }
    
    return inactive_problem

In [3]:
import math

def mod_ishigami(values, threshold, activated):
    """Modified Ishigami to have conditionally activated parameters.
    
    Based on the Ishigami test function as implemented in SALib (v1.2)
    
    Parameters
    ==========
    * values : np.array, of input values to the model
    * threshold : float, value which X_{1} should be greater than to activate parameters
    * activated : list, 1-based list of parameters to activate
    
    Returns
    ==========
    * np.array of results
    """
    Y = np.zeros([values.shape[0]])
    A = 7
    B = 0.1
    
    if isinstance(activated, int):
        active = [activated]

    for i, X in enumerate(values):
        Y[i] = math.sin(X[0]) + A * math.pow(math.sin(X[1]), 2) + \
            B * math.pow(X[2], 4) * math.sin(X[0])
        for pos in activated:
            val = X[pos-1]
            if X[0] >= threshold or val > 3.14:
                Y[i] = Y[i] + val

    return Y

The above represents:

$$
f(x)=\text{sin}(x_1)+a \text{sin}^2 (x_2)+bx_3^4 \text{sin}(x_1) + \gamma
$$

where $\gamma$ represents a conditionally active parameter(s),

$$
\gamma = 
\begin{cases}
    & \text{sum}(k) \text{ if } x_{1} > h \\
    & k_{i} \text{ if } k_{i} > 3.1 \\
    & 0 \text{ otherwise }
\end{cases}
$$

Here $k$ indicates the parameters that are conditional and $h$ is some arbitrary threshold; set to $2.0$ for this example

In [4]:
i_p = create_inactive(7)

In [5]:
# Generate samples
saltelli_w_inactive = saltelli.sample(i_p, 1, seed=SEED_VALUE)

# Run model (example)
# saltelli_Y = ishigami_wrapper(saltelli_w_inactive)
saltelli_Y = mod_ishigami(saltelli_w_inactive, 2.0, [5, 6, 7, 8, 9, 10])

Si = sobol.analyze(i_p, saltelli_Y, print_to_console=False)

total, first, second = Si.to_df()

first.round(4)

,S1,S1_conf
x1,0.2383,0.0
x2,2.3738,0.0
x3,-1.0164,0.0
x4,0.0000,0.0
x5,0.0000,0.0
x6,0.0000,0.0
x7,0.0000,0.0
x8,0.0000,0.0
x9,0.0000,0.0
x10,0.0000,0.0


Saltelli/Sobol method cannot reliably detect conditionally active or parameters that are active at the extremes with the same number of samples compared to traditional OAT

In [6]:
# Generate samples
saltelli_w_inactive = saltelli.sample(i_p, 1, seed=SEED_VALUE)

# Run model (example)
# saltelli_Y = ishigami_wrapper(saltelli_w_inactive)
saltelli_Y = mod_ishigami(saltelli_w_inactive, 2.0, [5, 6, 7, 8, 9, 10])

Si = sobol.analyze(i_p, saltelli_Y, print_to_console=False)

total, first, second = Si.to_df()

first.round(4)

,S1,S1_conf
x1,0.2383,0.0
x2,2.3738,0.0
x3,-1.0164,0.0
x4,0.0000,0.0
x5,0.0000,0.0
x6,0.0000,0.0
x7,0.0000,0.0
x8,0.0000,0.0
x9,0.0000,0.0
x10,0.0000,0.0


OAT cannot identify parameters that activate depending on another parameter (parameter interaction), but does detect them when they are expected to be active at their extremes

In [7]:
i_p = create_inactive(7)

In [8]:
t_oat_sample = traditional_oat_sample(i_p, 10)
Y = mod_ishigami(t_oat_sample, 2.0, [5, 6, 7, 8, 9, 10])
oat_analyze(i_p, t_oat_sample, Y).mean(axis=0)

array([ 0.03079795,  0.02152244, -0.29999674,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [9]:
t_oat_sample = oat_extremity_sample(i_p)
Y = mod_ishigami(t_oat_sample, 2.0, [5, 6, 7, 8, 9, 10])
oat_analyze(i_p, t_oat_sample, Y).mean(axis=0)

array([ 0.09761998, -0.0007076 ,  0.00986315,  0.        ,  0.51643876,
        0.51643876,  0.51643876,  0.51643876,  0.51643876,  0.51643876])